In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

import time

import pandas as pd

c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

In [15]:
# URL base
url = "https://www.tesla.com"

postal_zips = {
    "spain": {
        'cp': '45950',
        'url': 'es_ES'
    },
    "portugal": {
        'cp': '1000-004',
        'url': 'pt_PT'
    },
    "france": {
        'cp': '70123',
        'url': 'fr_FR'
    },
    "italy": {
        'cp': '10024',
        'url': 'it_IT'
    }
}



# Configuración del diccionario de modelos
cars_types = {
    'Model S': {
        'model': 'ms',
        'colors': ['BLACK','WHITE','BLUE','SILVER']
    },
    'Model 3': {
        'model': 'm3',
        'colors': ['BLACK','WHITE','BLUE','GREY','SILVER','CHERRY']
    },
    'Model X': {
        'model': 'mx',
        'colors': ['BLACK','WHITE','BLUE','GREY','SILVER','CHERRY']
    },
    'Model Y': {
        'model': 'my',
        'colors': ['BLACK','WHITE','BLUE','SILVER','CHERRY']
    }
}


types = {
    'Nuevo': 'new',
    'Ocasion': 'used'
}


In [16]:
# Diccionario para almacenar los datos
data = []
for country in postal_zips:
    for type in types:
        for car_type in cars_types:
            for color in cars_types[car_type]['colors']:
                current_car = cars_types[car_type]['model']
                current_type = types[type]
                current_country = postal_zips[country]


                # Navegar a la página del inventario del modelo específico
                # params = f'PAINT={color}&arrangeby=plh&zip={current_country["cp"]}&range=0'
                params = f'PAINT={color}&arrangeby=plh&range=0'
                c_driver.get(f'{url}/{current_country["url"]}/inventory/{current_type}/{current_car}?{params}')
                time.sleep(10)  # Esperar para que cargue la página

                # Buscar los coches en la página
                cars = c_driver.find_elements(By.XPATH, '/html/body/div[1]/div/div[1]/main/div/article')
                for car in cars:
                    
                    traction = car.find_element(By.XPATH, './section[1]/div[1]/div[1]').text
                    
                    if current_type == 'new':
                        price = car.find_element(By.XPATH, './section[1]/div[2]/div[1]/div/span[1]').text
                        year = datetime.now().year
                    else:
                        price = car.find_element(By.XPATH, './section[1]/div[2]/div/div/span').text
                        year = car.find_element(By.XPATH, './section[1]/div[1]/h3/span[1]').text
                    
                    try:
                        km = car.find_element(By.XPATH, './section[1]/div[1]/div[2]/span').text or 0
                    except NoSuchElementException:
                        km = 0

                    range_ = car.find_element(By.XPATH, './section[5]/div/ul/li[1]/div[1]/span[1]').text
                    max_vel = car.find_element(By.XPATH, './section[5]/div/ul/li[2]/div[1]/span[1]').text
                    zero_hundred = car.find_element(By.XPATH, './section[5]/div/ul/li[3]/div[1]/span[1]').text

                    # Agregar los datos al diccionario
                    data.append({
                        'sell_type': type,
                        'car_type': car_type,
                        'year': year,
                        'color': color,
                        'country': country,
                        'traction': traction,
                        'price': price,
                        'km': km,
                        'range': range_,
                        'max_vel': max_vel,
                        'zero_hundred': zero_hundred
                    })

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./section[1]/div[1]/div[1]"}
  (Session info: chrome=131.0.6778.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF676706CC5+28821]
	(No symbol) [0x00007FF676673850]
	(No symbol) [0x00007FF67651578A]
	(No symbol) [0x00007FF6765691BE]
	(No symbol) [0x00007FF6765694AC]
	(No symbol) [0x00007FF67655C52C]
	(No symbol) [0x00007FF67658F33F]
	(No symbol) [0x00007FF67655C3F6]
	(No symbol) [0x00007FF67658F510]
	(No symbol) [0x00007FF6765AF412]
	(No symbol) [0x00007FF67658F0A3]
	(No symbol) [0x00007FF67655A778]
	(No symbol) [0x00007FF67655B8E1]
	GetHandleVerifier [0x00007FF676A3FCCD+3408029]
	GetHandleVerifier [0x00007FF676A5743F+3504143]
	GetHandleVerifier [0x00007FF676A4B61D+3455469]
	GetHandleVerifier [0x00007FF6767CBDCB+835995]
	(No symbol) [0x00007F

In [12]:
df = pd.DataFrame(data)
df

,sell_type,car_type,year,color,country,traction,price,km,range,max_vel,zero_hundred
0,Nuevo,Model S,2024,BLACK,spain,"Model S, tracción a las cuatro ruedas",90.770 €,Cuentakilómetros con 551 km,634,250,"3,2"
1,Nuevo,Model S,2024,BLACK,spain,Plaid,101.480 €,Cuentakilómetros con 3652 km,600,322,"2,1"
2,Nuevo,Model S,2024,BLACK,spain,Plaid,102.540 €,Cuentakilómetros con 1700 km,600,322,"2,1"
3,Nuevo,Model S,2024,WHITE,spain,"Model S, tracción a las cuatro ruedas",82.010 €,Cuentakilómetros con 2622 km,634,250,"3,2"
4,Nuevo,Model S,2024,WHITE,spain,"Model S, tracción a las cuatro ruedas",99.600 €,Cuentakilómetros con 503 km,634,250,"3,2"
...,...,...,...,...,...,...,...,...,...,...,...
115,Nuevo,Model 3,2024,SILVER,spain,Gran autonomía con tracción integral,50.380 €,Cuentakilómetros con 13 km,629,201,"4,4"
116,Nuevo,Model 3,2024,SILVER,spain,Gran autonomía con tracción integral,50.760 €,0,629,201,"4,4"
117,Nuevo,Model 3,2024,SILVER,spain,Gran autonomía con tracción integral,52.110 €,0,629,201,"4,4"
118,Nuevo,Model 3,2024,SILVER,spain,Gran autonomía con tracción integral,52.360 €,0,629,201,"4,4"


In [8]:
df.isna().sum()

sell_type       0
car_type        0
year            0
traction        0
price           0
km              0
range           0
max_vel         0
zero_hundred    0
dtype: int64

In [9]:
df.to_csv('tesla.com.csv')

In [10]:
# 06 / 12 / 2024